# Logistic Regression

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#%matplotlib inline
#%matplotlib notebook
plt.rcParams["figure.figsize"] = (10,6)
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
pd.set_option('display.float_format', lambda x: '%.3f' % x)
#pd.options.display.float_format = '{:.3f}'.format

Bu data setinde en iyi treshold'u seçmeyi öğreneceğiz. Elimizde dengesiz bir data seti var, skorları nasıl iyileştirebiliriz bunu göreceğiz.

In [2]:
df=pd.read_csv("diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.600,0.627,50,1
1,1,85,66,29,0,26.600,0.351,31,0
2,8,183,64,0,0,23.300,0.672,32,1
3,1,89,66,23,94,28.100,0.167,21,0
4,0,137,40,35,168,43.100,2.288,33,1


Pregnancies              : Kaç defa hamile kalındığı

Glucose                  : Vücuttaki şeker oranı

BloodPressure            : Tansiyon

SkinThickness            : Deri kalınlığı

Insulin                  : Vücudun ürettiği insulin oranı

BMI                      : Vücut indexi

DiabetesPedigreeFunction : Ailede şeker hastalığı olup olmaması durumuna göre skorlar

Outcome : 1--> Şeker hastası, 0---> Şeker hastası değil

In [4]:
df.shape

(768, 9)

## Exploratory Data Analysis and Visualization

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.000,3.845,3.370,0.000,1.000,3.000,6.000,17.000
Glucose,768.000,120.895,31.973,0.000,99.000,117.000,140.250,199.000
BloodPressure,768.000,69.105,19.356,0.000,62.000,72.000,80.000,122.000
SkinThickness,768.000,20.536,15.952,0.000,0.000,23.000,32.000,99.000
Insulin,768.000,79.799,115.244,0.000,0.000,30.500,127.250,846.000
BMI,768.000,31.993,7.884,0.000,27.300,32.000,36.600,67.100
DiabetesPedigreeFunction,768.000,0.472,0.331,0.078,0.244,0.372,0.626,2.420
Age,768.000,33.241,11.760,21.000,24.000,29.000,41.000,81.000
Outcome,768.000,0.349,0.477,0.000,0.000,0.000,1.000,1.000


In [7]:
df.Outcome.value_counts()    # 1 sayısı az görünüyor ama skorlara bakmadan 'dengesizlik var' gibi bir yorum yapmıyoruz.

0    500
1    268
Name: Outcome, dtype: int64

In [8]:
sns.countplot(df.Outcome);

In [9]:
sns.boxplot(df.Pregnancies);    

In [10]:
# df=df[df.Pregnancies<13]   Outlier değerler gerçekte de olabilir. 17 kere hamile kalan kadınlar var. 
# Gerçek dünya verileri olduğu için tutuyoruz ama sayıları az olduğu için atıladabilir.

In [11]:
sns.boxplot(df.SkinThickness);

In [12]:
df=df[df.SkinThickness<70]   # Gerçekte 100 diye bir deri kalınlığı olmadığı için onu attık.

In [13]:
sns.boxplot(df.SkinThickness);

In [14]:
sns.boxplot(df.Insulin);

In [15]:
sns.boxplot(df.Glucose);

In [16]:
df=df[df.Glucose>0]    # Glukoz 0 olamaz o yüzden attık.

In [17]:
sns.boxplot(df.Glucose);

In [18]:
sns.boxplot(df.BloodPressure);

In [19]:
df=df[df.BloodPressure>35]    # Kan basıncı 30'un altında olamaz, o yüzden 35 altını attık.

In [20]:
sns.boxplot(df.BloodPressure);

In [21]:
sns.boxplot(df.BMI);

In [22]:
df=df[df.BMI>0]    # Vücut kitle indexi 0 olamaz, o yüzden attık.

In [23]:
sns.boxplot(df.BMI);

In [24]:
df.shape

(720, 9)

In [25]:
df.Outcome.value_counts()     # Bazı verileri attıktan sonra veri sayımız biraz düştü.

0    473
1    247
Name: Outcome, dtype: int64

In [26]:
index = 0
plt.figure(figsize=(20,20))
for feature in df.columns:
    if feature != "Outcome":
        index += 1
        plt.subplot(3,3,index)
        sns.boxplot(x='Outcome',y=feature,data=df)

In [27]:
plt.figure(figsize=(10,8))                     # Multicollineraity olsa bile Ridge ve Lasso arka planda bu sorunu giderecek (Default--> Ridge)
sns.heatmap(df.corr(), annot=True);     

In [28]:
# df.corr()                                                         # 1 sınıfıyla olan corr'ların görseli.
# df.corr()["Outcome"].sort_values().plot.barh()                    # En yüksek corr ilişkisi glukoz ile.
df.corr()["Outcome"].drop("Outcome").sort_values().plot.barh();     # Glukoz, kilo(BMI), Age yüksekse şeker hastası olma ihtimali yüksek.

In [29]:
sns.pairplot(df, hue = "Outcome");

## Train | Test Split and Scaling

In [30]:
X=df.drop(["Outcome"], axis=1)
y=df["Outcome"]

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

Datada dengesizlik olduğu düşünülüyorsa hem test hem de train seti split işleminde eşit oranlarla dağılsın diye 'statify = y' denir.Bu şekilde 0 sınıfının da %20'sini 1 sınıfının da %20'sini test için ayırır.

In [33]:
scaler = StandardScaler()

In [34]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Modelling

In [35]:
from sklearn.linear_model import LogisticRegression

In [36]:
log_model=LogisticRegression()

In [37]:
log_model.fit(X_train_scaled, y_train)

LogisticRegression()

In [38]:
y_pred=log_model.predict(X_test_scaled)

In [39]:
y_pred_proba = log_model.predict_proba(X_test_scaled)

pred'leri karşılaştırmak için X_test ve y_test'leri birleştirip pred ve pred_probayı feature olarak ekledik.

pred_proba'da 0.5'in üstündekileri 1'e altındakileri 0'a atadığını görüyoruz.

In [40]:
test_data = pd.concat([X_test, y_test], axis=1)
test_data["pred"] = y_pred
test_data["pred_proba"] = y_pred_proba[:,1]
test_data.sample(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,pred,pred_proba
68,1,95,66,13,38,19.600,0.334,25,0,0,0.029
260,3,191,68,15,130,30.900,0.299,34,0,1,0.767
512,9,91,68,0,0,24.200,0.200,58,0,0,0.119
94,2,142,82,18,64,24.700,0.761,21,0,0,0.261
425,4,184,78,39,277,37.000,0.264,31,1,1,0.784
413,1,143,74,22,61,26.200,0.256,21,0,0,0.204
433,2,139,75,0,0,25.600,0.167,29,0,0,0.198
174,2,75,64,24,55,29.700,0.370,33,0,0,0.045
282,7,133,88,15,155,32.400,0.262,37,0,0,0.376
520,2,68,70,32,66,25.000,0.187,25,0,0,0.017


## Model Performance on Classification Tasks

In [41]:
from sklearn.metrics import confusion_matrix, classification_report

Hem train hem de test setini aynı anda görmek için aşağıdaki fonksiyonu yazdık. Amacımız, train ile test datalarındaki skorları kıyaslayarak bir overfitting veya underfitting durumu var mı bunu tespit etmek.

In [42]:
def eval_metric(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    
    print("Test_Set")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print()
    print("Train_Set")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))

df' de 0 ve 1 sınıfları arasında unbalance bir durum olabileceğini gözlemlemiştik. Aşağıdaki skorlara baktığımızda bunu teyit edebiliyoruz, skorlar kötü.

Train_Set ve Test_Set skorları birbirine yakın olduğu için overfitting bir durumdan da bahsedemeyiz.

In [43]:
eval_metric(log_model, X_train_scaled, y_train, X_test_scaled, y_test)

Test_Set
[[85 10]
 [20 29]]
              precision    recall  f1-score   support

           0       0.81      0.89      0.85        95
           1       0.74      0.59      0.66        49

    accuracy                           0.79       144
   macro avg       0.78      0.74      0.75       144
weighted avg       0.79      0.79      0.79       144


Train_Set
[[337  41]
 [ 89 109]]
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       378
           1       0.73      0.55      0.63       198

    accuracy                           0.77       576
   macro avg       0.76      0.72      0.73       576
weighted avg       0.77      0.77      0.77       576



0 sınıfına ait skorların daha iyi, 1 sınıfına ait skorların daha kötü olduğunu gözlemliyoruz. Peki bunun sebebi ne?

Çünkü 0 sınıfına ait gözlem sayısı daha fazla. Bu yüzden eğitimini daha iyi yapmış.

Cross Validate ile yukarıda aldığımız skorları teyit edeceğiz :

In [44]:
from sklearn.model_selection import cross_validate

In [45]:
model = LogisticRegression()

In [46]:
scores = cross_validate(model, X_train_scaled, y_train, scoring = ['precision','recall','f1','accuracy'], cv = 10)

In [47]:
df_scores = pd.DataFrame(scores, index = range(1, 11))
df_scores                                                  # Binary modellerdeki skorlar her zaman 1 class'ına ait skorlardır.

,fit_time,score_time,test_precision,test_recall,test_f1,test_accuracy
1,0.004,0.004,0.600,0.450,0.514,0.707
2,0.002,0.003,0.643,0.450,0.529,0.724
3,0.003,0.003,0.923,0.600,0.727,0.845
4,0.003,0.002,0.857,0.600,0.706,0.828
5,0.002,0.003,0.706,0.600,0.649,0.776
6,0.004,0.002,0.647,0.550,0.595,0.741
7,0.003,0.002,0.714,0.526,0.606,0.772
8,0.003,0.002,0.647,0.579,0.611,0.754
9,0.002,0.003,0.733,0.550,0.629,0.772
10,0.002,0.003,0.625,0.500,0.556,0.719


In [48]:
df_scores.mean()[2:]     # Sadece skorları görebilmek için 2. indexten sonrasına bakıyoruz.   (Scale edilmiş skorlar)

test_precision   0.710
test_recall      0.541
test_f1          0.612
test_accuracy    0.764
dtype: float64

Aşağıya eval_metric' i tekrar yazdıralım, scale edilmeden önceki skorlar(yukarıdaki) ile sonraki skorları (aşağıdaki) kıyaslayalım.

Scale edildikten sonra skorların biraz düştüğünü gördük. Ama çok bariz bir fark yok diyebiliriz. Cross Validate işlemi bu durumu tespit etmek adına önemli.

In [49]:
eval_metric(log_model, X_train_scaled, y_train, X_test_scaled, y_test)               # (Scale edilmemiş skorlar)

Test_Set
[[85 10]
 [20 29]]
              precision    recall  f1-score   support

           0       0.81      0.89      0.85        95
           1       0.74      0.59      0.66        49

    accuracy                           0.79       144
   macro avg       0.78      0.74      0.75       144
weighted avg       0.79      0.79      0.79       144


Train_Set
[[337  41]
 [ 89 109]]
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       378
           1       0.73      0.55      0.63       198

    accuracy                           0.77       576
   macro avg       0.76      0.72      0.73       576
weighted avg       0.77      0.77      0.77       576



Yukarıdaki skorlarımızda Recall çok düşük. Amacımız, Recall'ı artırmak ama Precision ile dengeli bir şekilde. Dolayısıyla f1-score' da artmış olacak. Modelimiz iyileşecek.

!!!!!!!  Çok dengesiz datasetlerinde Test_Set ile Train_Set arasında çok fazla fark olduğunda overfitting durumu var diyemeyiz. Bu durumda bakacağımız skorlar 'macro' ve 'weighted' olmalı. Overfitting olup olmadığına bu iki skorla karar verebiliriz.  !!!!!!!

Modelimizde overfitting olsa bile logisticRegression() içine penalty = l1 gibi değerler yazarak overfitting ile mücadele edebiliriz. (Default değeri l2.)

## Cross Validate for 0 class

Eğer default class olan 1 değil de 0 sınıfı için cross validate yapmak istersek 'make_scorer' fonksiyonunu import ediyoruz.

In [50]:
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [51]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

Yukarıdaki skorları direk make_scorer içinde kullanamıyorum çünkü kullandığım zaman 1 sınıfının skorlarını hesaplıyor. Biz ise 0 sınıfının skorlarını istiyoruz.

make_scorer : 'pos_label' aslında make_scorer' ın içinde geçmiyor ama make_scorer içinde kullanılan fonksiyonun da içinde geçenleri kullanabilirsin diyor, böyle bir esneklik sağlıyor. Mesela f1_score()'un içinde geçen pos_label' ı burada kullanabileceğiz. Bu yüzden make_scorer'ın içine pos_label' ı ekledik.

In [52]:
f1_0 = make_scorer(f1_score, pos_label =0)
precision_0 = make_scorer(precision_score, pos_label =0)
recall_0 = make_scorer(recall_score, pos_label =0)

In [53]:
model = LogisticRegression()

cross_validate'de scoring'lere mse, rmse gibi skorları yazıyorduk. Burda 0 sınıfına ait skorları istediğimiz için, yukarıda tanımladığımız f1, precision ve recall değerlerini dict olarak cross_validate'in içine veriyoruz. Böylece 0 sınıfına ait skorları alabileceğiz. 

In [54]:
scores = cross_validate(model, X_train_scaled, y_train,scoring = {"precision_0":precision_0, "recall_0":recall_0, "f1_0":f1_0}, cv = 10)

Bulduğumuz skorları DataFrame yapısına çevirdik :

In [55]:
df_scores = pd.DataFrame(scores, index = range(1, 11))
df_scores

,fit_time,score_time,test_precision_0,test_recall_0,test_f1_0
1,0.006,0.003,0.744,0.842,0.790
2,0.004,0.006,0.750,0.868,0.805
3,0.003,0.003,0.822,0.974,0.892
4,0.004,0.005,0.818,0.947,0.878
5,0.002,0.003,0.805,0.868,0.835
6,0.003,0.002,0.780,0.842,0.810
7,0.006,0.003,0.791,0.895,0.840
8,0.004,0.002,0.800,0.842,0.821
9,0.007,0.002,0.786,0.892,0.835
10,0.003,0.002,0.756,0.838,0.795


In [56]:
df_scores.mean()[2:]         # Yukarıda bulduğumuz skorların ortalamasını aldık.

test_precision_0   0.785
test_recall_0      0.881
test_f1_0          0.830
dtype: float64

Aşağıda Cross_Validate yapılmamış değerleri tekrar yazdırdık ki bir kıyas yapalım, yukarıda yeni bulduğum değerlerle cross işlemi öncesi skorlarım değişmiş mi?

Kıyaslama için bu sefer 0 değerlerine odaklanacağız. Çünkü 0 değerlerinin scorlarını bulduk. (cross_validate işleminden sonra skorlarımın biraz düştüğünü gözlemliyorum.)

In [57]:
eval_metric(log_model, X_train_scaled, y_train, X_test_scaled, y_test)

Test_Set
[[85 10]
 [20 29]]
              precision    recall  f1-score   support

           0       0.81      0.89      0.85        95
           1       0.74      0.59      0.66        49

    accuracy                           0.79       144
   macro avg       0.78      0.74      0.75       144
weighted avg       0.79      0.79      0.79       144


Train_Set
[[337  41]
 [ 89 109]]
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       378
           1       0.73      0.55      0.63       198

    accuracy                           0.77       576
   macro avg       0.76      0.72      0.73       576
weighted avg       0.77      0.77      0.77       576



## GridSearchCV

Skorlarımız çok iyi çıkmadı. Peki bunları nasıl iyileştireceğiz?

In [58]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [59]:
from sklearn.model_selection import GridSearchCV

In [60]:
model = LogisticRegression()

Logistic Regression İçindeki Parametreler :

LogisticRegression overfitting ile mücadele etmek amacıyla içine penalty = l1, l2, elasticnet parametrelerini alıyordu. 

l2 ----> Ridge

l1 ----> Lasso

Linear Regression'daki alpha yerine burda C parametresi var. Bu parametre alpha ile ters orantılı çalışır. Alpha büyüdükçe regularization artar; C küçüldükçe regularization artar (bias ekler). Yani C değerinin küçülmesi iyi bir şey.

class_weight : Class sayıları arasında dengesizlik varsa; sayısı az olan sınıfı daha çok ağırlıklandırır. Yani zayıf olan sınıfa daha çok tahmin yaptırır.

solver : Modeller metricleri minimize etmek için 'Gradient Descent tabanlı' çalışırlar. Solver metrikleri de Gradient Descent methodlarıdır. Çok bilinmiyorsa default değerlerinin değiştirilmesi önerilmez. Çoğunlukla default değeri iyi sonuç verir.   (solver : 'lbfgs')

Eğer data küçükse ''solver : liblinear'', çok büyük datalarda ise ''solver : sag'' veya ''solver : saga'' iyi bir seçim olabilir. Kafamızda soru işareti oluştuğu zaman bunları deneyerek sonuçları karşılaştırabiliriz.

multi_class : 0, 1, 2 diye üç sınıf olsun. ROC/AUC çizerken 2 sadece binary olanları çizebiliyor. Burdaki 3 sınıfı çizmek için herhangi bir sınıfı alıp geri kalanına tek bir sınıf gibi davranır. Böylece 2 sınıf varmış gibi olur. Tüm ihtimaller için bunu yapar ve çizgilerini çizer. multi_class = 'ovr' bunu sağlar. default = 'auto'

Biz aşağıda bir fonksiyon tanımlayarak Ridge ve Lasso'dan hangisinin daha iyi sonuç verdiğine bakacağız :

In [61]:
penalty = ["l1", "l2"]                # l1 ve l2 skorlarına bakacağız.
C = np.logspace(-1, 5, 20)            # C parametresi logspace aralığında daha iyi sonuçlar verir. (Hangi sayının logunu aldığımda bu aralıktan bir sayı döndürür?)
class_weight= ["balanced", None]      # Classlar arası dengeleme yapsın veya yapmasın.

# The "balanced" mode uses the values of y to automatically adjust weights inversely proportional to class frequencies 
# in the input data

solver = ["lbfgs", "liblinear", "sag", "saga"]   # Gradient descent methodlarından hangisini kullanayım?

In [62]:
param_grid = {"penalty" : penalty,
              "C" : C,
              "class_weight":class_weight,
              "solver":solver}


grid_model = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=10,
                          scoring = "recall",      # 1 sınıfına ait en iyi recall'ı hangi parametreler getirecek? Bunu hesaplar. 
                          n_jobs = -1)             # Recall dedik çünkü skorlarımızda bu değer kötü. f1 de diyebilirdik. Sırayla denenebilir.

In [63]:
grid_model.fit(X_train_scaled,y_train)     # Eğitimimizi yaptık.

GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.00000000e-01, 2.06913808e-01, 4.28133240e-01, 8.85866790e-01,
       1.83298071e+00, 3.79269019e+00, 7.84759970e+00, 1.62377674e+01,
       3.35981829e+01, 6.95192796e+01, 1.43844989e+02, 2.97635144e+02,
       6.15848211e+02, 1.27427499e+03, 2.63665090e+03, 5.45559478e+03,
       1.12883789e+04, 2.33572147e+04, 4.83293024e+04, 1.00000000e+05]),
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2'],
                         'solver': ['lbfgs', 'liblinear', 'sag', 'saga']},
             scoring='recall')

In [64]:
grid_model.best_params_        # Yukarıda tanımlanan modele göre çıkan en iyi parametre değerleri.

{'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}

eval_metric' i tekrar yazdırıp önceki değerleri ile kıyaslayalım :

In [65]:
eval_metric(grid_model, X_train_scaled, y_train, X_test_scaled, y_test)

Test_Set
[[76 19]
 [13 36]]
              precision    recall  f1-score   support

           0       0.85      0.80      0.83        95
           1       0.65      0.73      0.69        49

    accuracy                           0.78       144
   macro avg       0.75      0.77      0.76       144
weighted avg       0.79      0.78      0.78       144


Train_Set
[[288  90]
 [ 49 149]]
              precision    recall  f1-score   support

           0       0.85      0.76      0.81       378
           1       0.62      0.75      0.68       198

    accuracy                           0.76       576
   macro avg       0.74      0.76      0.74       576
weighted avg       0.78      0.76      0.76       576



Yeni sonuç ile eski sonucu kıyasladığımızda; precision değerinin düştüğünü ama recall değerinin de yükseldiğini görüyoruz. f1 score da 66'dan 70'e çıkarak dengeyi korumuş. Amacımıza ulaştık; recall değerini dengeli bir şekilde artırdık.

0 skorları ise düştü. Bizim amacımız hasta olanları yani 1'leri tespit etmek. Bu yüzden 1 olanları iyileştirmeye yönelik parametreler kullandık. 

Tek bir modelde hem 1 hem 0' lar için skorlara bakılmaz, bu hatalı olur. 0 skorlarına bakıyorsak ayrı model, 1 skorlarına bakıyorsak ayrı model kullanmalıyız. 

## ROC (Receiver Operating Curve) and AUC (Area Under Curve)

ROC/AUC; birçok treshold değeri belirler ve buna göre eksende noktalar bulur. (Treshold = 0.5'e göre düşman olduğunu bildim veya bilemedim gibi.) Bu noktaların altında kalan alan ne kadar büyükse, model dost ile düşmanı ayırmakta o kadar başarılı demektir.

1 sınıfını düşman, 0 sınıfını dost gibi düşünüyoruz ve amacımız düşmanı tespit etmek.

y ekseni, düşman olarak doğru tahmin ettiklerimiz.  (True Positive Rate)

x ekseni, düşman olarak yanlış tahmin ettiklerimiz. (False Positive Rate)

https://towardsdatascience.com/calculating-and-setting-thresholds-to-optimise-logistic-regression-performance-c77e6d112d7e

In [66]:
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve, roc_auc_score, auc, roc_curve, average_precision_score, precision_recall_curve

1 noktası True-Positive'in en yüksek olduğu nokta; 0 noktası ise False- Positive'in en düşük olduğu nokta. Amacımız ilkini yüksek, ikinciyi düşük yapmak ki alttaki alan büyüsün.

In [67]:
plot_roc_curve(grid_model, X_test_scaled, y_test);                # Modelin başarısı : 0.85

__Precision-Recall-Curve__

1 noktası Precision'in en yüksek olduğu nokta; 1 noktası ise Recall'in en yüksek olduğu nokta. Amacımız ikisini de yüksek tutmak ki alttaki alan büyüsün, model başarısı artsın. (Dengesiz datasetlerinde kullanılır.)

In [68]:
plot_precision_recall_curve(grid_model, X_test_scaled, y_test);      #Modelin başarısı : 0.76

Bizim için geçerli olan yöntem : precision_recall_curve. Çünkü datasetimiz dengesiz. (İlk yöntem daha iyi olmasına rağmen ikinci yöntemi seçtik.)

## Finding Best Threshold

Amacımız yeni treshold'lar belirleyerek en büyük alanı çizebilmek ve model başarısını artırabilmek.

default olarak treshold değeri = 0.5

Yeni tresholdlar belirleyerek mesela 0.3; 0.3 ün altındakilere 0; üstündekilere 1 diyeceğiz.

ROC ve AUC ile Best Treshold :

Bunun için en iyi treshold değerini bulacağız yani alttaki alanın en büyük olduğu treshold değeri.

!!!!!!       Dengeli datasetlerinde ROC / AUC, dengesiz datasetlerinde Precision Recall Curve kullanılır.   !!!!!!!!

!!!!!!!!! Best treshold sadece train setinde bulunur. Eğer test setinde de denersek 'data leakage(kopye)' olur.   !!!!!!!!

In [70]:
plot_roc_curve(grid_model, X_train_scaled, y_train);   

In [71]:
y_pred_proba = log_model.predict_proba(X_train_scaled)  # Train setindeki predict_proba' yı aldık ki yukardaki grafikteki skorla karşılaştırabilelim.
roc_auc_score(y_train, y_pred_proba[:,1])           # roc_ouc_score içine eğittiğimiz y yi ve y_train'den aldığımız proba'nın 1 sınıfı için olan değerlerini verdik.

0.8378493934049489

Değerimiz yukarıdaki grafikte 0.84 çıkmıştı predict işleminde de 0.83 çıktı. Birbirine yakın değerler elde ettik.

fp_rate : False - Positive Rate (Amaç minimum yapmak). (FPR)

tp_rate : True - Positive Rate (Amaç maximum yapmak).  (TPR)

treshold : 0 - 1 arasında aldığı olasılıklar.

In [72]:
fp_rate    # Her bir treshold'a göre aldığı olasılık değerleri.

NameError: name 'fp_rate' is not defined

In [73]:
tp_rate     #Her bir treshold'a göre aldığı olasılık değerleri.

NameError: name 'tp_rate' is not defined

In [74]:
fp_rate, tp_rate, thresholds = roc_curve(y_train, y_pred_proba[:,1])

(max TPR) - (min FPR) çıkarırsak; burası düşmanın en iyi tespit edildiği noktadır.

(Düşmana düşman dediğim max değerden, dosta düşman dediğim min değeri çıkardım.)

In [75]:
optimal_idx = np.argmax(tp_rate - fp_rate)          # İçerideki max değer neyse onun index nosunu döndürür.
optimal_threshold = thresholds[optimal_idx]         # Bulunan indexi tresholdun içine verdik. En optimal treshold'u bize döndürür.
optimal_threshold

0.33938184887578743

!!!!! Best treshold için ROC ve AUC da kullanabiliriz, Precision-Recall-Curve da kullanabiliriz. Aynı sonuçlar çıkar, sadece hesaplamaları farklı. (ROC AUC mantığı daha kolay) !!!!!

In [ ]:
Precision-Recall-Curve ile Best Treshold :

In [ ]:
plot_precision_recall_curve(grid_model, X_train_scaled, y_train);   

In [ ]:
y_pred_proba = log_model.predict_proba(X_train_scaled)
average_precision_score(y_train, y_pred_proba[:,1])

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train, y_pred_proba[:,1])

In [ ]:
optimal_idx = np.argmax((2 * precisions * recalls) / (precisions + recalls))
optimal_threshold = thresholds[optimal_idx]
optimal_threshold

In [ ]:
grid_model.predict_proba(X_test_scaled)[:,1]    # 0.5 treshold'a göre dönen değerler.

Biz aşağıdaki fonksiyonda artık 0.5 değil de yeni bulduğumuz best treshold olan 0.33'e göre değerler döndüreceğiz.

Önce seri içine yazdık yoksa apply fonk. uygulayamazdık.

In [ ]:
Aldığımız değer yeni treshold'dan büyükse (0.33), 1 sonucunu döndür; değilse 0 döndür.

In [ ]:
y_pred2 = pd.Series(grid_model.predict_proba(X_test_scaled)[:,1]).apply(lambda x : 1 if x >= optimal_threshold else 0)

In [ ]:
print(confusion_matrix(y_test,y_pred2))
print(classification_report(y_test,y_pred2))

Yukarıdaki sonuçlara baktığımızda 1 class'ına ait precision  değerleri hemen hemen aynı ama recall değeri baya yükseldi.

Aşağıda yeni treshold değeri ile train setine de baktık. Orda da 1 sınıfına ait recall değerlerinin iyileştiğini görüyoruz.

In [ ]:
y_train_pred2 = pd.Series(grid_model.predict_proba(X_train_scaled)[:,1]).apply(lambda x : 1 if x >= optimal_threshold else 0)
print(confusion_matrix(y_train, y_train_pred2))
print(classification_report(y_train, y_train_pred2))

Aşağıdaki fonksiyon, treshold'u ile oynanmış bir dataya Cross Validation'ın arkada yaptığı işlemin manual olarak yapılması. LogisticRegression'da yapılan işlemleri içeriyor :

In [ ]:
from sklearn.model_selection import StratifiedKFold    # Modeli kaç parçaya ayırmak istiyorsak ona göre index numaraları belirler.

def CV(n, est, X, y, optimal_threshold):
    skf = StratifiedKFold(n_splits = n, shuffle = True, random_state = 42)
    acc_scores = []
    pre_scores = []
    rec_scores = []
    f1_scores  = []
    
    X = X.reset_index(drop=True)       # Index no'ları her işlemden sonra sıfırlaması için.
    y = y.reset_index(drop=True)
    
    for train_index, test_index in skf.split(X, y):
        
        X_train = X.loc[train_index]
        y_train = y.loc[train_index]
        X_test = X.loc[test_index]
        y_test = y.loc[test_index]
        
        
        est = est
        est.fit(X_train, y_train)
        y_pred = est.predict(X_test)
        y_pred_proba = est.predict_proba(X_test)
             
        y_pred2 = pd.Series(y_pred_proba[:,1]).apply(lambda x : 1 if x >= optimal_threshold else 0)
        
        acc_scores.append(accuracy_score(y_test, y_pred2))
        pre_scores.append(precision_score(y_test, y_pred2, pos_label=1))
        rec_scores.append(recall_score(y_test, y_pred2, pos_label=1))
        f1_scores.append(f1_score(y_test, y_pred2, pos_label=1))
    
    print(f'Accuracy {np.mean(acc_scores)*100:>10,.2f}%  std {np.std(acc_scores)*100:.2f}%')
    print(f'Precision-1 {np.mean(pre_scores)*100:>7,.2f}%  std {np.std(pre_scores)*100:.2f}%')
    print(f'Recall-1 {np.mean(rec_scores)*100:>10,.2f}%  std {np.std(rec_scores)*100:.2f}%')
    print(f'F1_score-1 {np.mean(f1_scores)*100:>8,.2f}%  std {np.std(f1_scores)*100:.2f}%')

In [ ]:
model = LogisticRegression(C= 0.1, class_weight= 'balanced',penalty= 'l1',solver= 'liblinear')  
CV(10, model, pd.DataFrame(X_train_scaled), y_train, optimal_threshold)   

# Bulduğumuz C değeri ve kullandığımız parametreler neyse yazmalıyız.
# Scale edilmiş data array' e dönüştüğü için burda tekrar DataFrame'e dönüştürüyoruz.

n_split : Data setini 10'a böl 9' unu train, 1' ini test seti yapar. 9 tane train'in indexlerini belirler. 

test_index : Test seti için ayırdıklarının index no'su.

Bu index no'lara göre yukarıdaki fonksiyondaki for döngüsüne girer. Index no' lara göre X_train, y_train, X_test, y_test değerlerini belirler. Her for döngüsünde train ve test setleri değişir.

est = est kısmında modeli eğitir ve pred ve predict_proba' ları alır. 

y_pred2 = pd.Series(y_pred_proba[:,1]).apply(lambda x : 1 if x >= optimal_threshold else 0) kısmında ise daha önce yukarda yaptığımız gibi optimal treshold değerlerini bulur.

Bulduğu her değeri acc_scores = [],
    pre_scores = [],
    rec_scores = [],
    f1_scores  = [] içine atar.

print kısmında ise bulunan değerlerin ortalamasını alır.

## Final Model and Model Deployment

In [76]:
scaler = StandardScaler().fit(X)

In [77]:
import pickle
pickle.dump(scaler, open("scaler_diabates", 'wb'))

In [78]:
X_scaled = scaler.transform(X)

In [79]:
final_model = LogisticRegression(class_weight = "balanced").fit(X_scaled, y)

In [80]:
pickle.dump(final_model, open("final_model_diabates", 'wb'))

In [81]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,720.000,3.881,3.366,0.000,1.000,3.000,6.000,17.000
Glucose,720.000,121.897,30.649,44.000,100.000,117.000,142.000,199.000
BloodPressure,720.000,72.589,12.075,38.000,64.000,72.000,80.000,122.000
SkinThickness,720.000,21.288,15.477,0.000,0.000,24.000,33.000,63.000
Insulin,720.000,84.676,117.277,0.000,0.000,48.000,132.000,846.000
BMI,720.000,32.424,6.842,18.200,27.475,32.350,36.600,67.100
DiabetesPedigreeFunction,720.000,0.475,0.333,0.078,0.245,0.378,0.629,2.420
Age,720.000,33.339,11.737,21.000,24.000,29.000,41.000,81.000


In [82]:
my_dict = {"Pregnancies": [3, 6, 5],
           "Glucose": [117, 140, 120],
           "BloodPressure": [72, 80, 75],
           "SkinThickness": [23, 33, 25],
           "Insulin": [48, 132, 55],
           "BMI": [32, 36.5, 34],
           "DiabetesPedigreeFunction": [0.38, 0.63, 0.45],
           "Age": [29, 40, 33]
          }

In [83]:
sample = pd.DataFrame(my_dict)
sample

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3,117,72,23,48,32.000,0.380,29
1,6,140,80,33,132,36.500,0.630,40
2,5,120,75,25,55,34.000,0.450,33


In [84]:
scaler_diabates = pickle.load(open("scaler_diabates", "rb"))

In [85]:
sample_scaled = scaler_diabates.transform(sample)
sample_scaled

array([[-0.26176089, -0.15989353, -0.04880222,  0.1107269 , -0.3129508 ,
        -0.06203757, -0.28511694, -0.3699221 ],
       [ 0.63004277,  0.59105285,  0.61417137,  0.75730737,  0.40380099,
         0.59612131,  0.46610236,  0.56790858],
       [ 0.33277488, -0.06194401,  0.19981287,  0.240043  , -0.25322148,
         0.23047749, -0.07477553, -0.02889276]])

In [86]:
final_model = pickle.load(open("final_model_diabates", "rb"))

In [87]:
predictions = final_model.predict(sample_scaled)
predictions_proba = final_model.predict_proba(sample_scaled)
predictions2 = [1 if i >= optimal_threshold else 0 for i in predictions_proba[:,1]]

In [88]:
sample["pred_proba"] = predictions_proba[:,1]
sample["pred_0.50"] = predictions
sample["pred_0.34"] = predictions2
sample

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,pred_proba,pred_0.50,pred_0.34
0,3,117,72,23,48,32.000,0.380,29,0.327,0,0
1,6,140,80,33,132,36.500,0.630,40,0.772,1,1
2,5,120,75,25,55,34.000,0.450,33,0.481,0,1
